In [14]:
from neo4j import GraphDatabase
import uuid

# Neo4j connection details
# NEO4J_URI = "neo4j+s://http://localhost:7474" # ssl

# Connect to DB here.
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "1982"

# Neo4j driver initialization
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Neo4jClient class definition
class Neo4jClient:
    def __init__(self):
        self.driver = driver

    def close(self):
        self.driver.close()

    def add_entity(self, entity_type, entity):
        with self.driver.session() as session:
            session.execute_write(self._add_entity_tx, entity_type, entity)

    @staticmethod
    def _add_entity_tx(tx, entity_type, entity):
        query = (
            f"CREATE (e:{entity_type} {{id: $id}}) "
            f"SET e += $properties "
            "RETURN e"
        )
        tx.run(query, id=entity['id'], properties=entity)

    def create_relationships(self, entity_type, entity):
        with self.driver.session() as session:
            session.execute_write(self._create_relationships_tx, entity_type, entity)

    @staticmethod
    def _create_relationships_tx(tx, entity_type, entity):
        if entity_type == "Student":
            for course_code in entity.get('enrolled_courses', []):
                tx.run(
                    "MATCH (s:Student {id: $student_id}), (c:Course {course_code: $course_code}) "
                    "MERGE (s)-[:ENROLLED_IN]->(c)",
                    student_id=entity['id'], course_code=course_code
                )
        elif entity_type == "Course":
            tx.run(
                "MATCH (p:Professor {name: $instructor}), (c:Course {id: $course_id}) "
                "MERGE (p)-[:TEACHES]->(c)",
                instructor=entity['instructor'], course_id=entity['id']
            )

    def execute_query(self, query):
        with self.driver.session() as session:
            return session.run(query).data()

# Example Usage
if __name__ == "__main__":
    client = Neo4jClient()

    # Add multiple courses
    courses = [
        {
            "id": str(uuid.uuid4()),
            "name": "Introduction to AI",
            "course_code": "AI101",
            "department": "Computer Science",
            "instructor": "Dr. Jane Doe",
            "credits": 3
        },
        {
            "id": str(uuid.uuid4()),
            "name": "Data Structures",
            "course_code": "CS201",
            "department": "Computer Science",
            "instructor": "Dr. John Smith",
            "credits": 4
        }
    ]

    for course in courses:
        client.add_entity("Course", course)

    # Add multiple students
    students = [
        {
            "id": str(uuid.uuid4()),
            "name": "Alice Smith",
            "major": "Computer Science",
            "enrolled_courses": ["AI101", "CS201"],
            "gpa": 3.9,
            "year": "Sophomore"
        },
        {
            "id": str(uuid.uuid4()),
            "name": "Bob Johnson",
            "major": "Computer Science",
            "enrolled_courses": ["CS201"],
            "gpa": 3.5,
            "year": "Freshman"
        }
    ]

    for student in students:
        client.add_entity("Student", student)
        client.create_relationships("Student", student)

    # Query to fetch the relationships
    query = """
    MATCH (s:Student)-[:ENROLLED_IN]->(c:Course)
    RETURN s.name AS student_name, c.name AS course_name
    """
    result = client.execute_query(query)

    # Print query results
    for record in result:
        print(f"Student: {record['student_name']}, Course: {record['course_name']}")

    client.close()


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.CartesianProductWarning} {category: } {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (c))} {position: line: 1, column: 1, offset: 0} for query: 'MATCH (s:Student {id: $student_id}), (c:Course {course_code: $course_code}) MERGE (s)-[:ENROLLED_IN]->(c)'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.CartesianProductWarning} {category: } {title: This query builds a cartesian produc

Student: Alice Smith, Course: Introduction to AI
Student: Alice Smith, Course: Data Structures
Student: Alice Smith, Course: Introduction to AI
Student: Alice Smith, Course: Introduction to AI
Student: Bob Johnson, Course: Data Structures
